In [18]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

import warnings
warnings.simplefilter(action='ignore')

In [19]:
data = pd.read_csv('NQ.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')

In [20]:
data

,y,a1,a2,b1,b2,b3,b4,b5,b6,b7,...,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-09,0.000555,0.675009,0.67,2.029210,1.61,1.354201,0.94,2.40,18.70,80.56,...,-1.25,-2.029728,-1.92,1.65,72.17,38.37,-1.579,13.25,-3.08,-0.02
2020-01-10,-0.006665,0.012606,0.81,0.779468,2.12,0.766862,1.31,1.37,26.23,84.60,...,-0.80,-1.952923,-1.61,3.25,28.33,35.54,-1.813,13.51,-3.73,-0.95
2020-01-13,0.007542,-0.585729,-0.27,-0.086929,0.23,0.498800,0.50,1.42,-15.69,84.98,...,-2.30,-1.453885,-2.03,2.26,-1.09,28.82,-2.286,14.27,-5.64,-3.27
2020-01-14,-0.002971,0.653482,1.04,1.707098,2.14,1.053616,1.10,1.87,-14.73,88.06,...,0.66,-0.266841,-0.38,1.37,-51.36,32.75,-2.420,14.98,-4.62,-2.71
2020-01-15,0.001105,-0.207969,-0.24,-1.269340,-1.35,-1.061371,-1.11,1.73,35.38,76.77,...,2.32,2.254327,2.56,2.45,46.02,44.84,-2.038,14.63,-2.29,-0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-26,-0.000209,-0.984979,-0.93,-0.657744,-0.48,0.327235,0.45,2.86,-14.12,40.59,...,-1.37,-0.337247,-0.44,5.38,-11.75,34.64,-2.828,24.21,-9.41,-1.07
2024-07-29,-0.005282,0.149617,1.03,-0.338363,0.22,-0.487980,-0.81,1.60,-18.87,41.68,...,1.59,-1.367858,0.56,2.41,-31.09,38.36,-2.810,24.55,-7.78,0.26
2024-07-30,-0.016868,-0.425302,0.07,0.589971,0.13,1.015273,0.06,1.63,-13.03,42.38,...,-0.93,-0.758752,-1.00,3.89,-20.37,36.97,-2.702,24.56,-8.35,-0.96


In [21]:
# 用前一期填上空值
# df_fill = data.fillna(method='ffill')
df_fill = data.dropna()

df_features = df_fill.iloc[:,1:]
df_features = df_features.pct_change().dropna()
df_features.replace([np.inf,-np.inf],np.nan,inplace=True)
df_features = df_features.fillna(method='ffill')

df_clean = df_features.copy()
df_clean['y'] = df_fill['y']

In [22]:
df_dir = df_clean.copy()
for i in df_dir.columns:
    df_dir[i] = np.where(df_dir[i]<0,-1,1)
df_dir

,a1,a2,b1,b2,b3,b4,b5,b6,b7,b8,...,p3,p4,p5,p6,p7,p8,p9,p10,p11,y
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-10,-1,1,-1,1,-1,1,-1,1,1,1,...,-1,-1,1,-1,-1,1,1,1,1,-1
2020-01-13,-1,-1,-1,-1,-1,-1,1,-1,1,1,...,-1,1,-1,-1,-1,1,1,1,1,1
2020-01-14,-1,-1,-1,1,1,1,1,-1,1,1,...,-1,-1,-1,1,1,1,1,-1,-1,-1
2020-01-15,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,-1,-1,-1,1
2020-01-16,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-25,-1,1,-1,-1,-1,1,1,-1,-1,1,...,-1,-1,1,-1,-1,1,-1,1,-1,-1
2024-07-26,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,1,1,1,-1,-1
2024-07-29,-1,-1,-1,-1,-1,-1,-1,1,1,1,...,1,-1,-1,1,1,-1,1,-1,-1,-1


In [23]:
df_lead = pd.DataFrame()
for i in df_clean.columns:
    if i !='y':
        max_corr = 0
        best_shift = 0
        for j in range(0,252):
            df =pd.DataFrame()
            df['y'] = df_clean['y']
            df[i] = df_clean[i].shift(j)
            if abs(df.corr().loc['y',i])>abs(max_corr):
                max_corr = df.corr().loc['y',i]
                best_shift = j
        df_lead.loc[:,i] = [max_corr,best_shift]
df_lead       

,a1,a2,b1,b2,b3,b4,b5,b6,b7,b8,...,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11
0,0.096027,-0.101958,-0.085291,-0.087582,-0.13732,-0.088775,-0.100007,0.092289,0.112581,0.110871,...,-0.107211,-0.073289,0.074304,-0.105401,0.07369,-0.111268,-0.097862,0.099848,0.16425,0.129834
1,85.000000,9.000000,244.000000,226.000000,87.00000,10.000000,40.000000,170.000000,170.000000,31.000000,...,65.000000,231.000000,186.000000,227.000000,52.00000,187.000000,108.000000,251.000000,218.00000,214.000000


In [24]:
df_lead_dir = pd.DataFrame()

for i in df_dir.columns:
    if i !='y':
        max_corr = 0
        best_shift = 0
        for j in range(0,252):
            df =pd.DataFrame()
            df['y'] = df_dir['y']
            df[i] = df_dir[i].shift(j)
            if abs(df.corr().loc['y',i])>abs(max_corr):
                max_corr = df.corr().loc['y',i]
                best_shift = j
        df_lead_dir.loc[:,i] = [max_corr,best_shift]
df_lead_dir

,a1,a2,b1,b2,b3,b4,b5,b6,b7,b8,...,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11
0,0.079481,-0.098569,0.091979,0.092581,0.116944,-0.094625,0.099692,0.094214,-0.090646,0.100219,...,-0.11264,-0.083481,0.101568,-0.089827,-0.08231,-0.100867,-0.102678,0.105069,-0.091526,-0.096643
1,154.000000,191.000000,181.000000,10.000000,238.000000,250.000000,176.000000,225.000000,121.000000,206.000000,...,120.00000,196.000000,82.000000,227.000000,44.00000,89.000000,209.000000,149.000000,89.000000,73.000000


### **低相關性**

In [25]:
threshold = 0.0003
corr_matrix = df_clean.corr()
low_corr = []

for i in corr_matrix.columns.to_list():
    for j in corr_matrix.columns.to_list():
        if i!=j:
            if abs(corr_matrix.loc[i,j])<threshold:
                low_corr.append(j)
                low_corr.append(i)
low_corr = list(set(low_corr))
low_corr.remove('y')
print(len(low_corr))
print(low_corr)

147
['d3', 'p4', 'i2', 'f8', 'p5', 'i7', 'l6', 'n7', 'l8', 'b10', 'j3', 'f1', 'b9', 'h11', 'o1', 'c5', 'k10', 'n4', 'a2', 'a1', 'm1', 'l2', 'n8', 'n2', 'b8', 'i3', 'n3', 'p6', 'k11', 'e6', 'i10', 'l11', 'h2', 'n9', 'h8', 'p8', 'j4', 'c3', 'n10', 'b6', 'f9', 'i8', 'j8', 'i1', 'g5', 'm4', 'b4', 'c9', 'e1', 'i4', 'p7', 'o9', 'o10', 'k9', 'h7', 'd9', 'i6', 'c7', 'm11', 'n6', 'o3', 'k1', 'c2', 'g1', 'h10', 'j11', 'b1', 'k2', 'g11', 'k7', 'l4', 'j6', 'e11', 'g3', 'b3', 'c8', 'b11', 'h3', 'p10', 'f11', 'n11', 'g10', 'c11', 'f10', 'j10', 'g6', 'd1', 'd4', 'k8', 'c6', 'n1', 'l10', 'l5', 'd8', 'e8', 'h9', 'm5', 'm8', 'h4', 'd5', 'b7', 'j2', 'd2', 'o4', 'm2', 'h1', 'k6', 'b2', 'p2', 'b5', 'j7', 'j1', 'k3', 'e2', 'i5', 'm9', 'e4', 'l7', 'o11', 'o6', 'd6', 'o5', 'k4', 'd10', 'c10', 'm3', 'c4', 'l9', 'f7', 'l3', 'j9', 'i11', 'h6', 'p1', 'f2', 'e3', 'e10', 'g2', 'p11', 'g8', 'd11', 'm10', 'f3', 'i9', 'h5', 'f4', 'o2']


In [26]:
threshold = 0.0003
corr_matrix = df_dir.iloc[:,:-1].corr()
low_corr_dir = []

for i in corr_matrix.columns.to_list():
    for j in corr_matrix.columns.to_list():
        if i!=j:
            if abs(corr_matrix.loc[i,j])<threshold:
                low_corr_dir.append(j)
                low_corr_dir.append(i)
low_corr_dir = list(set(low_corr_dir))
# low_corr_dir.remove('y')
print(len(low_corr_dir))
print(low_corr_dir)

91
['p4', 'i2', 'f8', 'p5', 'l6', 'g9', 'n7', 'b10', 'j3', 'e7', 'h11', 'o1', 'n4', 'a1', 'n2', 'b8', 'p9', 'p3', 'n3', 'p6', 'k11', 'e6', 'l11', 'h2', 'j4', 'd7', 'g4', 'b6', 'f9', 'f6', 'g5', 'm4', 'b4', 'e1', 'p7', 'o9', 'k9', 'o10', 'c1', 'd9', 'i6', 'm11', 'n6', 'g1', 'j11', 'b1', 'k2', 'g11', 'k7', 'm6', 'j6', 'l4', 'e11', 'g3', 'b3', 'n11', 'g10', 'c11', 'd1', 'n1', 'l10', 'h9', 'm8', 'j2', 'm2', 'h1', 'k6', 'b2', 'b5', 'j7', 'j1', 'k3', 'e2', 'i5', 'l7', 'd6', 'k4', 'd10', 'm3', 'c10', 'c4', 'l3', 'p1', 'f2', 'e10', 'g2', 'p11', 'd11', 'f3', 'i9', 'o2']


### **Leading Factor**

In [27]:
num = 10
df_lead = df_lead.T
lead_factors = df_lead.sort_values(0,ascending=False).head(num).index.to_list()
lead_factors_days = df_lead.sort_values(0,ascending=False).head(num)[1].to_list()
print(lead_factors_days)
print(len(lead_factors))
print(lead_factors)


[139.0, 136.0, 218.0, 162.0, 91.0, 10.0, 187.0, 222.0, 200.0, 8.0]
10
['e8', 'h6', 'p10', 'm8', 'f3', 'e1', 'o4', 'm4', 'p1', 'm7']


### **Leading Factor by direction**

In [37]:
num = 10
df_lead_dir = df_lead_dir.T
lead_factors_dir = df_lead_dir.sort_values(0,ascending=False).head(num).index.to_list()
lead_factors_dir_days = df_lead.sort_values(0,ascending=False).head(num)[1].to_list()
print(len(lead_factors_dir))
print(lead_factors_dir)
print(lead_factors_dir_days)

10
['g8', 'h10', 'i11', 'h7', 'c8', 'b3', 'i10', 'c10', 'i2', 'n4']
[139.0, 136.0, 218.0, 162.0, 91.0, 10.0, 187.0, 222.0, 200.0, 8.0]


### **K-Means**

In [29]:
num = 30
df_kmeans = df_clean.copy()
x = df_kmeans.iloc[:,1:].T
model = KMeans(n_clusters=num, n_init='auto',random_state=1).fit(x)
df_grouped  = pd.DataFrame({'factor':x.index,'group':model.predict(x)})
# K-Means different group
k_means_pick = df_grouped.groupby('group').head(1)['factor'].to_list()
y_group = df_grouped[df_grouped['factor']=='y']['group'].values[0]
# K-Means same group with Y
k_means_y = df_grouped[df_grouped['group']==y_group]['factor'].to_list()
print(len(k_means_pick))
print(k_means_pick)
print(len(k_means_y))
print(k_means_y)
# df_grouped.groupby('group').count()

30
['a2', 'b3', 'c2', 'c3', 'c6', 'd1', 'd2', 'd6', 'e1', 'e3', 'e6', 'f3', 'g1', 'g6', 'h1', 'h3', 'h6', 'i6', 'i11', 'j3', 'j4', 'j6', 'k1', 'k6', 'l3', 'm3', 'm6', 'n3', 'p1', 'p6']
138
['a2', 'b1', 'b2', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'c1', 'c4', 'c5', 'c7', 'c8', 'c9', 'c10', 'c11', 'd3', 'd4', 'd5', 'd7', 'd8', 'd9', 'd10', 'd11', 'e2', 'e4', 'e5', 'e7', 'e8', 'e9', 'e10', 'e11', 'f1', 'f2', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'g2', 'g3', 'g4', 'g5', 'g7', 'g8', 'g9', 'g10', 'g11', 'h2', 'h4', 'h5', 'h7', 'h8', 'h9', 'h10', 'h11', 'i1', 'i2', 'i3', 'i4', 'i5', 'i7', 'i8', 'i9', 'i10', 'j1', 'j2', 'j5', 'j7', 'j8', 'j9', 'j10', 'j11', 'k2', 'k3', 'k4', 'k5', 'k7', 'k8', 'k9', 'k10', 'k11', 'l1', 'l2', 'l4', 'l5', 'l6', 'l7', 'l8', 'l9', 'l10', 'l11', 'm1', 'm2', 'm4', 'm5', 'm7', 'm8', 'm9', 'm10', 'm11', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9', 'o10', 'o11', 'p2', 'p3', 'p4

### **K-Means by direction**

In [30]:
num = 10
df_kmeans = df_dir.copy()
x = df_kmeans.iloc[:,1:].T
model = KMeans(n_clusters=num, n_init='auto',random_state=1).fit(x)
df_grouped  = pd.DataFrame({'factor':x.index,'group':model.predict(x)})
# K-Means different group
k_means_dir = df_grouped.groupby('group').head(1)['factor'].to_list()
y_group = df_grouped[df_grouped['factor']=='y']['group'].values[0]
# K-Means same group with Y
k_means_y_dir = df_grouped[df_grouped['group']==y_group]['factor'].to_list()
print(len(k_means_dir))
print(k_means_dir)
print(len(k_means_y_dir))
print(k_means_y_dir)
# df_grouped.groupby('group').count()

10
['a2', 'b3', 'b4', 'b5', 'b7', 'b8', 'b9', 'b10', 'c6', 'e7']
26
['b8', 'c8', 'd8', 'd10', 'e8', 'f8', 'f10', 'g8', 'h8', 'i8', 'i9', 'i10', 'j8', 'j10', 'k8', 'l8', 'l10', 'm8', 'm9', 'm10', 'n8', 'o8', 'o10', 'o11', 'p8', 'y']


### **多次同方向**

In [31]:
num=10
df_patern = df_dir.copy()
df_point = pd.DataFrame(columns=df_dir.columns[:-1])
for i in range(len(df_patern)):
    row = df_patern.iloc[i]
    for j in df_dir.columns.to_list()[:-1]:
        if row['y']==row[j]:
            df_point.loc[i,j]=1
        else:
            df_point.loc[i,j]=0
df_point.loc['Total'] = df_point.sum(axis=0)
df_point = df_point.T
df_point.sort_values('Total',axis=0,ascending=False)
similar_dir = df_point[1:].head(num).index.to_list()
# similar_dir.remove('y')
print(len(similar_dir))
print(similar_dir) 

10
['a2', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']


In [38]:
df_select_freature = {'lead_factor': lead_factors,
                      'lead_factors_days':lead_factors_days,
                      'lead_factor_dir': lead_factors_dir,
                      'lead_factor_dir_days': lead_factors_dir_days,
                      'k_means': k_means_pick,
                      'k_means_dir': k_means_dir,
                      'k_means_y': k_means_y,
                      'k_means_y_dir': k_means_y_dir,
                      'low_corr': low_corr,
                      'low_corr_dir':low_corr_dir,
                      'similar_dir':similar_dir
                      }
df_select_freature = pd.DataFrame.from_dict(df_select_freature, orient='index').transpose()
df_select_freature.to_csv('factor.csv',index=False)